# 0. Library Import & Dataset parsing

## Import library and data

In [1]:
pip install -q transformers

     |████████████████████████████████| 4.0 MB 30.6 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 27.7 MB/s 
     |████████████████████████████████| 895 kB 56.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.9 MB/s 


In [2]:
pip install -q datasets

     |████████████████████████████████| 325 kB 18.5 MB/s 
     |████████████████████████████████| 136 kB 40.0 MB/s 
     |████████████████████████████████| 1.1 MB 41.1 MB/s 
     |████████████████████████████████| 212 kB 58.3 MB/s 
     |████████████████████████████████| 127 kB 59.3 MB/s 
     |████████████████████████████████| 271 kB 43.9 MB/s 
     |████████████████████████████████| 144 kB 50.3 MB/s 
     |████████████████████████████████| 94 kB 965 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from transformers import pipeline
from datasets import load_dataset

In [4]:
sst2 = load_dataset('glue', 'sst2')

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

1. train: 67349
2. validation: 872
3. test: 1821


In [5]:
sst2

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

## Data Preprocessing

In [14]:
import math

In [20]:
train_small = sst2["train"].shuffle(seed=1002).select([i for i in list(range(int(len(sst2["train"])*0.1)))])
valid_small = sst2["validation"].shuffle(seed=1002).select([i for i in list(range(int(len(sst2["validation"])*0.1)))])
test_dataset = sst2["test"].shuffle(seed=1002).select([i for i in list(range(int(len(sst2["test"])*0.1)))])

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-98c11ee4a0e89078.arrow


# 1. Using the best benchmark

reference : https://paperswithcode.com/sota/sentiment-analysis-on-sst-2-binary

In [11]:
sst2["train"]
sst2["validation"]
sst2["test"]

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

# Model

## 1. T5-3B

In [6]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

In [7]:
from transformers import AutoTokenizer, AutoModelWithLMHead
#checkpoint = "t5-3b" - too much space
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelWithLMHead.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/10.6G [00:00<?, ?B/s]

KeyboardInterrupt: ignored

## 2. Transformer